# Laboratoria 9: BERT i atencja


### Zadanie 1 (3 pkt), atencja dekodera względem (en)kodera

Poniżej znajdują się dwie macierze, `encoder_states` oraz `decoder_states` reprezentujące stan warstwy ukrytej po przetworzeniu każdego slowa z enkodera i dekodera. Pojedynczy stan warstwy ukrytej zawiera embedding o dlugosci = 3. W enkoderze mamy 4 stany warstwy ukrytej RNNów, gdyż przetwarzamy sekwencję 4 tokenów.

W dekoderze mamy 5 tokenów, które powinny być wygenerowane z sekwencji przetwarzanej (en)koderem.

Zadanie polega na:
a) Obliczniu podobieństwa wszystkich embeddingów z dekodera (queries) względem wszystkich embeddingów kolejnych stanów (en)kodera (keys) [pamiętajcie, że macierze potrafią w transponowanie. W `NumPy` macierz transponujemy za pomocą `macierz.T`]

b) Na utworzonej macierzy podobieństwa należy wykonać softmax (zaimportowany z scipy). Uwaga:  pamiętajcie, żeby aplikować softmax w dobrym wymiarze. Wszystkie stany ukryte enkodera powinny zostac zasoftmaksowane względem zadanego stanu dekodera, nie odwrotnie. W scipy, funkcja softmax zawiera argument axis, który może pomóc.

c) Należy wykorzystać macierz atencji z kroku b) i `encoder_states` do wygenerowania macierzy zawierającej wektory kontekstu dla każdego tokenu z dekodera.


In [1]:
import numpy as np
from scipy.special import softmax

# scipy.special.softmax(x, axis=None)

encoder_states = np.array(
    [[1.2, 3.4, 5.6],    # embedding z warstwy ukrytej enkodera w kroku 1,  np. dla slowa Ala
    [-2.3, 0.2, 7.2],   # embedding z warstwy ukrytej enkodera w kroku 2,  np. dla slowa ma
    [10.2, 0.2, 0.3],   # embedding z warstwy ukrytej enkodera w kroku 3,  np. dla slowa kota
    [0.4, 0.7, 1.2]]    # embedding z warstwy ukrytej enkodera w kroku 4,  np. dla tokenu <EOS> (koniec sekwencji)
)



decoder_states = np.array(
    [[0.74, 0.23, 0.56],  # embedding z warstwy ukrytej dekodera w kroku 1,  np. przed wygenerowaniem slowa Alice
    [7.23, 0.12, 0.55],  # embedding z warstwy ukrytej dekodera w kroku 2,  np. przed wygenerowaniem slowa owns
    [9.12, 4.23, 0.44],  # embedding z warstwy ukrytej dekodera w kroku 3,  np. przed wygenerowaniem slowa a
    [4.1, 3.23, 0.5],    # embedding z warstwy ukrytej dekodera w kroku 4,  np. przed wygenerowaniem slowa cat
    [5.2, 3.1, 8.5]]     # embedding z warstwy ukrytej dekodera w kroku 5,  np. przed wygenerowaniem slowa cat
)

probabilities = decoder_states.dot(encoder_states.T)

print(f"a) Probabilities: \n {probabilities}")

softmaxResult = softmax(probabilities, axis=1)
print(f"b) Softmax: \n{softmaxResult}")

print(f"c) Context: \n{softmaxResult.dot(encoder_states)}")


a) Probabilities: 
 [[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]]
b) Softmax: 
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]
c) Context: 
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]


**Oczekiwane wartości:**

a) 
[[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]] 


b) 
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]] 

c) 
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]
 
 (albo to samo transponowane)


## Zadanie 2 (2 punkty): tokenizacja tekstu 

Korzystając z biblioteki transformers (https://huggingface.co/transformers/) wczytaj tokenizator BERTa (BERT to już wytrenowany (pretrenowany) model, oparty o ideę transformera (a w zasadzie o jego enkoder)). Ponieważ model jest gotowy i można go wykorzystać do generowania embeddingów tokenów, ważnym jest, aby tokenizacja była przeprowadzona identycznie do tego jak podczas trenowania BERTa.

Wybierzmy pretrenowany tokenizator o nazwie `bert-base-uncased` i zobaczmy jaki będzie efekt tokenizacji na tekście zawartym w zmiennej `text_to_tokenize`.

Zwróć uwagę na to, że niektóre rzadkie słowa zostały podzielone na subtokeny -- zgodnie z algorytmem WordPiece jaki omawialiśmy na przedostatnim spotkaniu.


In [2]:
# Uruchom mnie proszę
!pip install transformers

In [3]:
from transformers import BertTokenizer
text_to_tokenize = "I've bought a new GPU last year it was GeForce RTX 3060"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer(text_to_tokenize)
print(f"tokens ids: \n {tokens['input_ids']} \n and coresponding tokens: \n {tokenizer.convert_ids_to_tokens(tokens['input_ids'])}")


tokens ids: 
 [101, 1045, 1005, 2310, 4149, 1037, 2047, 14246, 2226, 2197, 2095, 2009, 2001, 16216, 14821, 19387, 2595, 24622, 2692, 102] 
 and coresponding tokens: 
 ['[CLS]', 'i', "'", 've', 'bought', 'a', 'new', 'gp', '##u', 'last', 'year', 'it', 'was', 'ge', '##force', 'rt', '##x', '306', '##0', '[SEP]']


## Zadanie 3 (brak punktów):
Poniżej znajduje się kod wykorzystujący przygotowane wcześniej zmienne `tokenizer` i `tokens` i które dla każdego tokenu z tokens generuje embedding. W odróżnieniu od GloVe, te embeddingi są świadome kontekstu w jakim właśnie występują. 

In [4]:
from transformers import BertModel
import torch

model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)  
model.eval()  # nie chcemy trenowac modelu, tylko go wykorzystac

tokens_with_specials = tokens['input_ids']  # BERT wymaga specjalnych tokenów [CLS] na poczatku i [SEP] separaującego pary zdań (BERT jest trenowany parami zdań)
tokens_with_specials = tokenizer.convert_tokens_to_ids(tokens_with_specials)  # zamiana listy tokenow na listę identyfikatorów (liczb) ze slownika
tokens_tensor = torch.tensor([tokens_with_specials])  # zamiana na tensor, opakowanie w batch

segments = torch.tensor([[1] * len(tokens_with_specials)])  # wygeneruj maskę mówiącą o tym które tokeny nalezą do zdania 1, a ktore do 2. W naszym zadaniu wszystkie tokeny naleza do zdania 1

with torch.no_grad():
    outputs = model(tokens_tensor, segments)  # wygenerujmy embeddingi BERTem
    tokens_embeddings = outputs['last_hidden_state'][0]  # wez pierwszy batch danych i ostatnią warstwę
    print(tokens_embeddings.shape)  # 20x768, mamy 20 (sub)tokenów, (18 wlasciwych + cls + sep) i kazdy mapowany jest na wektor liczb o dlugosci 768
    print(tokens_embeddings[1])  # wez embedding pierwszego subtokenu z sekwencji (przeskakujemy CLS token)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([20, 768])
tensor([-2.4855e-01,  6.4821e-01, -4.4505e-01, -5.3101e-01, -3.7247e-01,
        -4.5230e-01,  7.0275e-01, -3.2836e-01,  4.3371e-02, -2.0077e-01,
         7.7430e-02,  1.2603e-02, -5.3415e-03,  2.3520e-02,  1.0070e+00,
        -2.1925e-01,  7.9879e-01, -6.1398e-01,  2.2324e-01, -1.2752e-01,
        -5.6451e-01, -2.5957e-01, -5.0997e-03,  6.4843e-01, -1.8477e-02,
        -5.4585e-01,  3.8544e-01, -8.5510e-01, -2.4844e-02, -2.6924e-01,
         1.7788e+00, -1.1623e+00, -5.5582e-01,  6.0392e-01, -3.9546e-01,
        -1.1233e+00,  5.5031e-01, -2.4410e-01,  1.5945e-01,  1.7484e-01,
        -9.7499e-01, -6.1897e-01,  1.7481e-01,  4.0425e-01, -7.5808e-01,
        -1.3810e+00,  2.1792e-01,  3.6299e-01,  5.7498e-01,  5.1727e-01,
         1.2242e-02,  2.7538e-01, -1.9612e-02,  4.8239e-01, -9.6976e-02,
         6.5601e-01, -1.9422e-01, -8.0068e-01, -5.9801e-01, -7.7547e-01,
         1.2948e+00, -8.7721e-02, -4.9269e-01, -2.1871e-01,  3.5231e-01,
         2.9646e-01, -2.7637e